# Каталоги Responsum
Пример ноутбука для работы с каталогами Responsum

### Зададим тип установки

In [ ]:
facility = 'mobile' # возможные значения: 'desktop', 'mobile', 'desktop_pre'

## Инициализация

При построении отчета первый шаг в любом ноутбуке - загрузка библиотек, которые помогут обращаться к API Responsum и работать с данными.

Выполните следующую ячейку, для этого перейдите в нее и нажмите Ctrl+Enter 

In [ ]:
%reload_ext autoreload
%autoreload 2

import os
import re
import json
import datetime
import time
import pandas as pd
from pathlib import Path
from IPython.display import JSON

from mediascope_api.core import net as msnet
from mediascope_api.responsum import catalogs as rc

# создаем объекты для работы с каталогами и заданиями,
# а так же загружаем каталоги
rcats = rc.ResponsumCats(facility)

print(f"Объектов в media-каталоге: {rcats.holdings.shape[0]}")

# Социально-демографические переменные

### Все социально-демографические переменные
Получим список всех социально-демографических переменных в WEB-Index

In [ ]:
demo_cat = rcats.get_demo()
demo_cat

Экспорт каталога в Excel. Файл сохраняется в директорию, в которой лежат текущие ноутбуки.

In [ ]:
writer = pd.ExcelWriter('demo_cat.xlsx')
demo_cat.to_excel(writer, 'demo_cat', index=False)
writer.save()

Экспорт каталога в csv. Файл сохраняется в директорию, в которой лежат текущие ноутбуки.

In [ ]:
demo_cat.to_csv('demo_cat.csv', encoding = 'utf-16')

### Поиск по социально-демографическим переменным

Найдем переменные, которые содержат текст "Численность"

In [ ]:
rcats.get_demo(find_text='численность')

# Работа с медиа-каталогом

## Получить медиа-каталог

Медиа-каталог загружается на шаге инициализации и доступен через DataFrame: __rcats.holdings__

In [ ]:
rcats.holdings

### Выгрузить медиа-каталог в Excel

Выгрузим медиа-каталог в Excel, но не на один лист, а каждую ветвь (holding, network, agency) - на отдельный лист .

Выгружать будем в папку с Excel: __../excel/media_catalog.xlsx__

In [ ]:
writer = pd.ExcelWriter('../excel/media_catalog.xlsx')
for b in rcats.holdings['branch'].drop_duplicates().to_list():
    rcats.holdings[rcats.holdings['branch'] == b].to_excel(writer, b, index=False)
writer.save()

## Поиск в медиа-каталоге

### Поиск идентификатора сайта avito.ru
Найдем все сайты, в названия которых входит текст "avito.ru" (поиск выполняется без учета регистра).

Искать будем только в названиях сайтов ветки "holding", для этого зададим 

```
branch = 'holding'
find_in = ['site']
```
<a id=’catalogs_site_avito’></a>

In [ ]:
site_avito = rcats.find_media('avito.ru', branch = 'holding', find_in=['site'])
site_avito

Из полученного результата видим, что нас интересует сайт с __ID = 15828__.

Экспорт каталога в Excel. Файл сохраняется в директорию, в которой лежат текущие ноутбуки.

In [ ]:
writer = pd.ExcelWriter('site_avito.xlsx')
site_avito.to_excel(writer, 'site_avito', index=False)
writer.save()

Экспорт каталога в csv. Файл сохраняется в директорию, в которой лежат текущие ноутбуки.

In [ ]:
site_avito.to_csv('site_avito.csv', encoding = 'utf-16')

### Поиск идентификатора сайта Ivi.ru
Найдем все сайты, в названия которых входит текст "ivi.ru" (поиск выполняется без учета регистра).

Искать будем только в названиях сайтов ветки "holding", для этого зададим 

```
branch = 'holding'
find_in = ['site']
```

<a id=’catalogs_site_ivi’></a>

In [ ]:
site_ivi = rcats.find_media('ivi.ru', branch = 'holding', find_in=['site'])
site_ivi

Из полученного результата видим, что нас интересует сайт с __ID = 15869__.

Экспорт каталога в Excel

In [ ]:
writer = pd.ExcelWriter('site_ivi.xlsx')
site_ivi.to_excel(writer, 'site_ivi', index=False)
writer.save()

Экспорт каталога в csv

In [ ]:
site_ivi.to_csv('site_ivi.csv', encoding = 'utf-16')

### Поиск идентификатора сайта Kommersant.ru
Найдем все сайты, в названия которых входит текст "kommersant.ru" (поиск выполняется без учета регистра).

Искать будем только в названиях сайтов ветки "holding", для этого зададим 

```
branch = 'holding'
find_in = ['site']
```

In [ ]:
site_kommersant = rcats.find_media('kommersant.ru', branch = 'holding', find_in=['site'])
site_kommersant.head()

Из полученного результата видим, что нас интересует сайт с __ID = 118__.

### Поиск идентификатора сайта Vk.com
Найдем все сайты, в названия которых входит текст "vk.com" (поиск выполняется без учета регистра).

Искать будем только в названиях сайтов ветки "holding", для этого зададим 

```
branch = 'holding'
find_in = ['site']
```

In [ ]:
site_vk = rcats.find_media('Vk.com', branch = 'holding', find_in='site')
site_vk

Из полученного результата видим, что нас интересует сайт с __ID = 16571__.

### Поиск идентификатора сайта Odnoklassniki.ru
Найдем все сайты, в названия которых входит текст "odnoklassniki.ru" (поиск выполняется без учета регистра).

Искать будем только в названиях сайтов ветки "holding", для этого зададим 

```
branch = 'holding'
find_in = ['site']
```

In [ ]:
holding_ok = rcats.find_media('Odnoklassniki.ru', branch = 'holding', find_in='site')
holding_ok

Из полученного результата видим, что нас интересует сайт с __ID = 12808__.

### Поиск идентификатора холдинга Google
Найдем все холдинги, в названия которых входит текст "google" (поиск выполняется без учета регистра).

Искать будем только по ветке "holding", для этого зададим 

```
branch = 'holding'
find_in = ['holding']
```

In [ ]:
holding_ok = rcats.find_media('google', branch='holding', find_in=['holding'])
holding_ok

### Поиск идентификатора бренда для рекламных кампаний
Также можно найти идентификатор нужного бренда.
Для рекламных кампаний соответствие следующее:

- Holding - AdAgency
- Site - Brand
- Section - Subbrand
- Subsection - Position

Искать ID бренда будем только в поле "site" (brand) ветки agency 

```
branch = 'agency'
find_in = ['site']
```

In [ ]:
# шаблон запроса на поиск идентификатора бренда
brand_x = rcats.find_media(find_text='VI', branch = 'agency', find_in=['site'])
brand_x

### Пример поиска названия сайта по ID 

В примере ниже найдем название сайта по его ID 16571 - это Vk.com


In [ ]:
rcats.find_media('16571', find_in=['site'])